# Application logs in Synapse Spark

By default, Spark adds 1 record to the MDC (Mapped Diagnostic Context): mdc.taskName, which shows something like task 1.0 in stage 0.0. You can add %X{mdc.taskName} to your patternLayout in order to print it in the logs. Moreover, you can use spark.sparkContext.setLocalProperty(s"mdc.$name", "value") to add user specific data into MDC. The key in MDC will be the string of “mdc.$name”.

https://spark.apache.org/docs/latest/configuration.html#configuring-logging

https://docs.microsoft.com/en-us/azure/synapse-analytics/spark/apache-spark-azure-log-analytics

## Log4J

### Loggers/Cetoggories & Levels

In [7]:
import org.apache.log4j._
import org.apache.log4j.spi._

val logRepo:LoggerRepository = LogManager.getLoggerRepository

import scala.collection.JavaConversions._
val currentLoggers=logRepo.getCurrentLoggers().asInstanceOf[java.util.Enumeration[Logger]].toSet

println(s"Total loggers: ${currentLoggers.size}")

currentLoggers.foreach{
  it => {
    println(s"Logger: ${it.getName}, Level enabled: ${it.getLevel}, DEBUG enabled? ${it.isDebugEnabled}")
  }
}

### Log Levels

By default all **DEBUG** level is disabled for all loggers in Synapse Spark. Hence the DEBUG logs are not propogated to the Log Analytics workspace.

NOTE: INFO is set as the default log level in Synapse spark. Because DEBUG has lower priority than INFO (`DEBUG < INFO < WARN < ERROR < FATAL`), all debug logs are not outputted.

In [8]:
//Pay attemtion to the Total Loggers count from previous cell output. It is 427
//After this cell, the count should be 428
val logger = org.apache.log4j.LogManager.getLogger("com.aravind.SparkAppOne")
logger.info("info message 1")
logger.debug("debug message 1")
logger.warn("warn message 1")
logger.error("error message 1")
logger.fatal("fatal message 1")

In [9]:
import scala.collection.JavaConversions._
val currentLoggers=logRepo.getCurrentLoggers().asInstanceOf[java.util.Enumeration[Logger]].toSet

println(s"Total loggers: ${currentLoggers.size}")
currentLoggers.filter(l=> l.getName.contains("aravind")).foreach(it => {
    println(s"Logger: ${it.getName}, Level enabled: ${it.getLevel}, DEBUG enabled? ${it.isDebugEnabled}")
  })

### Enable DEBUG

However, you would need DEBUG enabled on DEV and TEST environments and in PROD while debugging a specific issues. Spark uses Log4j for all logging. It provides a mechanism to set custome log levels programatically as shown below.

```
val myLogger = LogManager.getLogger("com.aravind.SparkAppOne")
myLogger.setLevel(Level.DEBUG)
```

In [10]:
println("Log level BEFORE changing it")
currentLoggers.filter{_.getName.contains("com.aravind")}.foreach{
  it => {
    println(s"Logger: ${it.getName}, Level enabled: ${it.getLevel}, DEBUG enabled? ${it.isDebugEnabled}")
  }
}

println("Log level AFTER changing it")
val aravindLogger = LogManager.getLogger("com.aravind.SparkAppOne")
aravindLogger.setLevel(Level.DEBUG)

currentLoggers.filter{_.getName.contains("com.aravind")}.foreach{
  it => {
    println(s"Logger: ${it.getName}, Level enabled: ${it.getLevel}, DEBUG enabled? ${it.isDebugEnabled}")
  }
}

logger.debug("DEBUG msg after enabling DEBUG level")
logger.info("INFO msg after enabling DEBUG level")

### Appenders

In [11]:
val rootLogger = LogManager.getRootLogger

import scala.collection.JavaConversions._
val allAppenders = rootLogger.getAllAppenders().asInstanceOf[java.util.Enumeration[Appender]].toSet

println(s"Total Appenders: ${allAppenders.size}")

allAppenders.foreach{
  it => {
    println(s"Appender: ${it.getName}, Layout: ${it.getLayout}, Filter: ${it.getFilter}")
    it.getLayout
  }
}

val laAppender = rootLogger.getAppender("LogAnalyticsAppender")
println("LogAnalyticsAppender: " +laAppender)

val laLayout = laAppender.getLayout
println("LogAnalyticsAppender.Layout: "+laLayout)
println("LogAnalyticsAppender.Layout contnetType: " +laLayout.getContentType)
println("LogAnalyticsAppender.Layout Header: " +laLayout.getHeader)
println("LogAnalyticsAppender.Layout Footer: " +laLayout.getFooter)

### Custom Layout

In [ ]:
//val PATTERN = "%d [%p|%c|%C{1}] %m%n";
val PATTERN = "%d{ISO8601} %-5p [%t] %c{4} - %m%n"
aravindAppender.setLayout(new com.microsoft.azure.synapse.loganalytics.logging.JSONLayout(false))
logger.info("INFO 1After setting new layout")
logger.debug("DEBUG 1After setting new layout")

In [ ]:
import com.microsoft.azure.synapse.loganalytics.logging._

val laAppender = rootLogger.getAppender("LogAnalyticsAppender")
val lal: JSONLayout = laAppender.getLayout().asInstanceOf[JSONLayout]
println(lal.getLocationInfo)
println(lal.getContentType)
println(lal.getJsonConfiguration)
println(lal.getHeader)
println(lal.getFooter)
//println(lal.format)
//println((lal.getContentType)
//println((lal.getHeader)
// println((lal.getFooter)
//println((lal.getJsonConfiguration)
// println((lal.getLocationInfo)

In [ ]:
spark.sparkContext.setLocalProperty("mdc.customMdcProp", "my value")
logger.info("Test custom MDC property")

### Exception on Driver

In [ ]:
throw new Exception("Exception from Driver")

In [ ]:
try {
    throw new Exception("Test Log error")
} catch {
    //case ex: Exception => logger.error(ex)
    case ex: Exception => logger.error(ex.getMessage, ex)
}

### Exception on Worker

In [ ]:
import org.apache.spark.sql.Row
import org.apache.spark.sql.types._

val numRdd = sc.parallelize(Seq(
    Row(1,1), Row(1,0), Row(2,0)
))

val schema = StructType(Array(
    StructField("a",DoubleType,true),
    StructField("b",DoubleType,true)
    ))

val numDf = spark.createDataFrame(numRdd, schema)
numDf.createOrReplaceTempView("number_tbl")

In [ ]:
numDf.rdd.getNumPartitions

In [ ]:
%%sql
SELECT a, b from number_tbl

## Reference

- [Log4j 1.2 Manual](https://logging.apache.org/log4j/1.2/manual.html)
- [Log4j 1.2 JavaDoc](https://logging.apache.org/log4j/1.2/apidocs/index.html)